## DLT Notebook - Gold Layer

In [0]:
looktables_rules = {
    "rule1" : "show_id is NOT NULL",
}

In [0]:
# Create a DLT table named "gold_netflixdirectors"
@dlt.table(name="gold_netflixdirectors")

@dft.expect_all_or_drop(looktables_rules)

def myfunc():
    # Read streaming data from Silver layer in ADLS Gen2
    df = spark.readStream.format("delta").load("abfss://silver@netflixprojectstorageacc.dfs.core.windows.net/netflix_directors")
    
    # Return DataFrame to be stored as a managed DLT table
    return df


In [0]:
@dlt.table(
    name = "gold_netflixcast"
)
@dft.expect_all_or_drop(looktables_rules)
def myfunc():
    df = spark.readStream.format("delta").load("abfss://silver@netflixprojectstorageacc.dfs.core.windows.net/gold_netflix_cast")
    return df

In [0]:
@dlt.table(
    name = "gold_netflixcountries"
)
@dft.expect_all_or_drop(looktables_rules)
def myfunc():
    df = spark.readStream.format("delta").load("abfss://silver@netflixprojectstorageacc.dfs.core.windows.net/netflix_countries")
    return df

In [0]:
@dlt.table(
    name = "gold_netflixcategory"
)
@dft.expect_or_drop("rule1" , "show_id is NOT NULL")
def myfunc():
    df = spark.readStream.format("delta").load("abfss://silver@netflixprojectstorageacc.dfs.core.windows.net/netflix_category")
    return df

In [0]:
@dlt.table
def gold_stg_netflixtitles():
    df = spark.readStream.format("delta").load("abfss://silver@netflixprojectstorageacc.dfs.core.windows.net/gold_netflixtitles")
    return df

In [0]:
from pyspark.sql.functions import *

In [0]:
@dlt.table
def gold_trns_netflixtitles():
    df = spark.readStream.table("LIVE.gold_stg_netflixtitles")
    df = df.withColumn("newflag",lit(1))
    return df

In [0]:
masterdata_rules = {
    "rule1" : "newflag is NOT NULL",
    "rule2" : "show_id is NOT NULL"
}

In [0]:
@dlt.table
@dft.expect_all_or_drop(masterdata_rules)
def gold_netflixtitles():
    df = spark.readStream.table("LIVE.gold_trns_netflixtitles")
    return df